# Analyzing the Mercari children's book dataset

### Import libraries and files

In [24]:
import os
import pandas as pd


In [37]:
os.chdir('/Users/mleong/github/used-books')
books = pd.read_csv('used-book-data.csv')
books.head()

,brand,category,condition,description,id,posted,price,seller_name,shipping,sold,title
0,NaN,"Other,Books,Children's books",New,Free with another book purchase,84914972647,07/10/17,$3,C Sells,$4.00 | from Texas,Buy now,Book
1,NaN,"Other,Books,Children's books",New,Book-When Bad Thinds Happen in God's Wonderful...,28411714700,07/15/17,$3,Investinyou,$4.00 | 3+ days | from Texas,Buy now,Book- When Bad Things Happen
2,NaN,"Other,Books,Children's books",New,This listing is for one book. Please let me kn...,21933816748,05/26/19,$11,HWalters55,Free | from Washington,Buy now,Usborne Wipe-Clean- NEW
3,NaN,"Other,Books,Children's books",New,Brand new. Never used. Ask me about bundling.,89750511304,11/11/17,$3,Bargain hauls,$4.00 | 3+ days | from Minnesota,Buy now,Jingle Bells Book
4,NaN,"Other,Books,Children's books",New,My Sister The Vampire Vampalicious. New softco...,496308655,08/19/16,$4,Dustyattic,Free | from Connecticut,Buy now,My Sister The Vampire


In [38]:
# Row count
rows = books.shape[0]
print("Number of rows: ", str(rows))

# Column count
columns = books.shape[1]
print("Number of columns: ", str(columns))

# Column names
print("Column names: ", str(books.columns))

# Indexing method
print("Index method: ", str(books.index))

# Data types of all columns
print("Data types for entire dataframe: ")
books.info()

# Data type of particular column
print("Data type of price column specifically is: ", str(books.price.dtype))



Number of rows:  49481
Number of columns:  11
Column names:  Index(['brand', 'category', 'condition', 'description', 'id', 'posted',
       'price', 'seller_name', 'shipping', 'sold', 'title'],
      dtype='object')
Index method:  RangeIndex(start=0, stop=49481, step=1)
Data types for entire dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49481 entries, 0 to 49480
Data columns (total 11 columns):
brand          25163 non-null object
category       49478 non-null object
condition      49481 non-null object
description    49481 non-null object
id             49481 non-null int64
posted         49481 non-null object
price          49481 non-null object
seller_name    49481 non-null object
shipping       49481 non-null object
sold           49481 non-null object
title          49481 non-null object
dtypes: int64(1), object(10)
memory usage: 4.2+ MB
Data type of price column specifically is:  object


### Clean-up time!

In [39]:
# eliminate duplicate rows
books = books.drop_duplicates()
books.shape[0]

22134

In [40]:
# id: convert to string
books.id = str(books.id)
print("Data type of id column specifically is: ", str(books.id.dtype))

Data type of id column specifically is:  object


In [41]:
# posted: convert to datetime
books.posted = pd.to_datetime(books.posted)
books['posted'].head()

0   2017-07-10
1   2017-07-15
2   2019-05-26
3   2017-11-11
4   2016-08-19
Name: posted, dtype: datetime64[ns]

In [42]:
# sold: combine buy now and add to cart categories to be 'for sale'
books = books.replace('Buy now', 'available')
books.sold.unique()

array(['available', 'SOLD'], dtype=object)

In [43]:
# price: modify to eliminate '$' sign and convert to number

books.price = books.price[1:-1]
books['price'].head()

0    NaN
1     $3
2    $11
3     $3
4     $4
Name: price, dtype: object

In [ ]:
# shipping: create new simpler column "location" that is just the state product is coming from

In [ ]:
# shipping: create new simpler column "free_shipping" to just be free or not free

In [ ]:
# shipping: create new column "shipping_fee" that is continuous variable from $0 to whatever shipping fee is

In [ ]:
# total price: new column that is sum of price and shipping fee